# MapEXIF-Colab
In Google Colab, read the EXIF information (GPS data) of images stored in a specific folder and visualize it on a map.

## Mount google drive
Mount your Google Drive where the images taken with your smartphone are stored.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set the Google Drive directory path

Specify the Google Drive directory path where your images are stored. In the example below, the path points to a folder named `example_dataset` inside `MyDrive`.

In [2]:
# Set the Google Drive directory path
img_directory = '/content/drive/MyDrive/example_dataset/'

## Install Folium＆Exifread
Install the two required libraries.

- [Folium](https://github.com/python-visualization/folium) is a Python library that makes it easy to create interactive maps using Leaflet.js, enabling visualization of geospatial data directly in Python notebooks.
- [Exifread](https://pypi.org/project/ExifRead/) is a Python library for extracting EXIF metadata, including GPS information, from image files.

In [3]:

!pip install folium
!pip install exifread

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.5 MB/s eta 0:00:00


## Defining the required Python functions

The following Python functions are defined to extract GPS information (latitude, longitude, and altitude) from the EXIF data of images taken with your smartphone.

In [4]:
import exifread

def read_exif_from_image(image_path):
    """Read EXIF data from an image file"""
    try:
        with open(image_path, "rb") as f:
            exif_data = exifread.process_file(f)
        return exif_data
    except FileNotFoundError:
        print(f"File not found: {image_path}")
        return None
    except IOError:
        print(f"Failed to read file: {image_path}")
        return None


def dms_to_decimal(dms):
    """Convert a Degree-Minute-Second (DMS) list to decimal coordinates"""
    degrees, minutes, seconds = dms
    return degrees + (minutes / 60) + (seconds / 3600)


def get_coordinates(exif_data):
    """Extract latitude and longitude from EXIF tags and return them as a tuple"""
    try:
        pre_latitude = eval(exif_data["GPS GPSLatitude"].printable)
        pre_longitude = eval(exif_data["GPS GPSLongitude"].printable)
        latitude = dms_to_decimal(pre_latitude)
        longitude = -1 * dms_to_decimal(pre_longitude)
        return (latitude, longitude)
    except KeyError:
        print("Required GPS information is missing.")
        return None

## Reading EXIF Data and Collecting GPS Information

This part of the code scans through all **HEIC images** in the specified Google Drive directory and attempts to read their EXIF metadata.  
For each image, the following information is extracted:  

- **File name** (without extension)  
- **GPS coordinates** (latitude and longitude)  

The extracted values are appended to separate lists (`name_list`, `location_list`) for later processing.  

If EXIF data cannot be read or GPS information is missing, the script prints a warning message instead of storing the data.

In [5]:
import glob
import os

# Extract GPS coordinates from HEIC images in the directory
name_list = []
location_list = []
for image_path in sorted(
    glob.glob(os.path.join(img_directory, '*.HEIC'))):

    exif_data = read_exif_from_image(image_path)
    if exif_data:
        coordinates = get_coordinates(exif_data)
        if coordinates:
            basename_without_ext = os.path.splitext(os.path.basename(image_path))[0]
            location_list.append(coordinates)
            name_list.append(basename_without_ext)
        else:
            print("Failed to retrieve GPS information.")
    else:
        print("Failed to read EXIF data.")

## Visualizing GPS Coordinates from HEIC Images

This Colab snippet extracts GPS coordinates from HEIC images in a specified directory and visualizes them on an interactive map using Folium.

In [20]:
import folium
import glob
import os

# Create a map with folium.Map() by specifying the center coordinates and zoom level
newmap = folium.Map(location=[37.8, -122.4], zoom_start=8)

# Extract GPS coordinates from HEIC images in the directory
for image_path in sorted(
    glob.glob(os.path.join(img_directory, '*.HEIC'))):
    exif_data = read_exif_from_image(image_path)
    if exif_data:
        coordinates = get_coordinates(exif_data)
        if coordinates:
            folium.Marker(location=(coordinates[0], coordinates[1]), popup=image_path).add_to(newmap)
        else:
            print("Failed to obtain GPS information.")
    else:
        print("Failed to read EXIF data.")

# Visualize a map with GPS coordinates
newmap